In [ ]:
%pip install torch transformers datasets tqdm
!mkdir checkpoints

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 55.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 69.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
%pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.1 MB/s eta 0:00:00


In [1]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import accuracy_score
from torch.utils.data import DataLoader
import json


# Load necessary packages
from dataclasses import dataclass
from typing import List, Tuple

from torch.utils.data import Dataset
from transformers import PreTrainedTokenizerFast
from datasets import load_dataset

import torch.nn as nn
from torch.optim import Optimizer, AdamW
from tqdm.notebook import tqdm

In [2]:
# from datasets import Dataset
import pickle

In [3]:
from transformers import pipeline
# from transformers.pipelines.pt_utils import KeyDataset

In [4]:
from datasets import load_dataset

# ds = load_dataset("ChilleD/StrategyQA")

In [4]:
model_name2 = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
model_name = "tiiuae/falcon-7b-instruct"

In [6]:
with open('/kaggle/input/strategyqa/train.json', 'r') as file:
    train_data = json.load(file)

# Print the data
print(len(train_data))

2061


In [5]:
with open('/kaggle/input/strategyqa/dev.json', 'r') as file:
    validation_data = json.load(file)

# Print the data
print(len(validation_data))

229


In [6]:
train_data[0]

{'qid': '2bc9c4f9c19c167187f2',
 'term': 'Genghis Khan',
 'description': 'founder and first Great Khan of the Mongol Empire',
 'question': 'Are more people today related to Genghis Khan than Julius Caesar?',
 'answer': True,
 'facts': ['Julius Caesar had three children.',
  'Genghis Khan had sixteen children.',
  'Modern geneticists have determined that  out of every 200 men today has DNA that can be traced to Genghis Khan.'],
 'decomposition': ['How many kids did Julius Caesar have?',
  'How many kids did Genghis Khan have?',
  'Is #2 greater than #1?'],
 'evidence': [[[['Caesarion-2', 'Julia (daughter of Caesar)-1']],
   [['Alakhai Bekhi-1', 'Tolui-1'], 'no_evidence'],
   ['operation']],
  [[['Julius Caesar-75']], [['Genghis Khan-17']], ['operation']],
  [[['Gaius Julius Caesar-7']],
   [['Genghis Khan-15'], 'no_evidence'],
   ['no_evidence', 'operation']]]}

In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [7]:
torch.manual_seed(0)

tokenizer = AutoTokenizer.from_pretrained(model_name)
pipe = pipeline(
    "text-generation",
    model=model_name,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

tokenizer_config.json:   0%|          | 0.00/1.13k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/17.7k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.48G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

Device set to use cuda:0


In [9]:
torch.manual_seed(1)

tokenizer = AutoTokenizer.from_pretrained(model_name2)
pipe = pipeline(
    "text-generation",
    model=model_name2,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

tokenizer_config.json:   0%|          | 0.00/3.07k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/679 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.55G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

Device set to use cuda:0


In [8]:
tokenizer.pad_token = tokenizer.eos_token

In [11]:
torch.manual_seed(1)
prompt = """Answer the question with 'Yes' or 'No'.
Question: Did Aristotle use a laptop?
Answer:
"""

sequences = pipe(
    prompt,
    max_new_tokens=15,
    return_full_text = False,
)

for seq in sequences:
    print(f"{seq['generated_text']}")

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


No


In [8]:
torch.manual_seed(1)
prompt = """Answer the question with 'Yes' or 'No'.
Question: Can a microwave melt a Toyota Prius battery?
Answer:
"""

sequences = pipe(
    prompt,
    max_new_tokens=15,
    return_full_text = False,
)

for seq in sequences:
    print(f"{seq['generated_text']}")

</think>

No, a microwave cannot melt a Toyota Prius battery.


In [30]:
torch.manual_seed(1)
prompt = """Briefly explain your reasoning, and then answer the question with 'Yes' or 'No'.
Question: Did Aristotle use a laptop?
Reasoning and Answer:
"""
prompt2 = """Let's think this through step by step.
Question: Did Aristotle use a laptop?
Answer:
"""


sequences = pipe(
    prompt2,
    do_sample=True,
    top_k=10,
    max_new_tokens=100,
    return_full_text = False,
)

"""
sequences = pipe(
    prompt2,
    max_new_tokens=100,
    return_full_text = False,
)
"""

for seq in sequences:
    print(f"{seq['generated_text']}")

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


1. Did the ancient Greeks even own a laptop as they did not have laptops back in the time of Aristotle?
2. Even if they did have laptops during the time of Aristotle, he lived in ancient Greece, not the present day where laptops are a common sight.

Conclusion: No, Aristotle did not have access to laptops or computers.


In [17]:
torch.manual_seed(1)
prompt = """Briefly explain your reasoning, and then answer the question with 'Yes' or 'No'.
Question: Did Aristotle use a laptop?
Reasoning and Answer:
"""
prompt2 = """Let's think this through step by step.
Question: Did Aristotle use a laptop?
Answer:
"""


sequences = pipe(
    prompt2,
    do_sample=True,
    top_k=10,
    max_new_tokens=100,
    return_full_text = False,
)

"""
sequences = pipe(
    prompt2,
    max_new_tokens=100,
    return_full_text = False,
)
"""

for seq in sequences:
    print(f"{seq['generated_text']}")

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Step 1: Identify the question: Did Aristotle use a laptop?
Step 2: Break it down into smaller parts.
Step 3: Analyze each part.
Step 4: Synthesize the information to form a coherent answer.
Step 5: Evaluate the answer for accuracy and completeness.
Step 6: Reflect on the process and improve if necessary.
Alright, let's apply this method to the question: Did Aristotle use a laptop?
First, I need to understand what the question is


In [155]:
train_data[128]

{'qid': 'a46d7c79c3cae661a145',
 'term': 'Haiku',
 'description': 'very short form of Japanese poetry',
 'question': 'Are most books written as a Haiku?',
 'answer': False,
 'facts': ['Haiku is a very short poem',
  'Haiku is written with 3 short phrases.'],
 'decomposition': ['What is the format of a haiku?',
  'Are chapter books written like #1?'],
 'evidence': [[[['Haiku-2']], ['operation']],
  [[['Haiku-2']], [['Chapter book-1']]],
  [[['Haiku-2']], ['no_evidence']]]}

In [43]:
torch.manual_seed(1)
prompt2 = """Let's think this through step by step. Decompose the question into a series of explicit sub-questions and answer each of them.
Example:
Question: Are more people today related to Genghis Khan than Julius Caesar?
Decomposition:
1. How many kids did Julius Caesar have?
Julius Caesar had three children.
2. How many kids did Genghis Khan have?
Genghis Khan had sixteen children.
3. Is #2 greater than #1?
Modern geneticists have determined that out of every 200 men today has DNA that can be traced to Genghis Khan.
Final Answer: Yes

Question: Can the President of Mexico vote in New Mexico primaries?
Decomposition:
"""

prompt = """Let's think this through step by step. Decompose the question into a series of explicit sub-questions and answer each of them.
Example 1:
Question: Are more people today related to Genghis Khan than Julius Caesar?
Decomposition:
1. How many kids did Julius Caesar have?
Julius Caesar had three children.
2. How many kids did Genghis Khan have?
Genghis Khan had sixteen children.
3. Is #2 greater than #1?
Final Answer: Yes

Example 2:
Question: Does a Generation Y member satisfy NYPD police officer age requirement?
Decomposition:
1. How old do you have to be to be an NYPD officer?
The NYPD has a minimum age requirement of 21.
2. How old are Generation Y members currently?
Members of Generation Y were born between 1980 and 1994.
3. Is #2 higher than #1?
Final Answer: Yes

Example 3:
Question: Is shrimp scampi definitely free of plastic?
Decomposition:
1. What protein is Shrimp scampi made out of?
Shrimp scampi is a dish made with shrimp.
2. What have #1 been found to contain?
Shrimp have been found to contain microplastics.
3. Are #2 free from plastic?
Microplastics are plastic material.
Final Answer: No

Question: Are most books written as a Haiku?
Decomposition:
"""

"""
Example 2:
Question: Is shrimp scampi definitely free of plastic?
Decomposition:
1. What protein is Shrimp scampi made out of?
Shrimp scampi is a dish made with shrimp.
2. What have #1 been found to contain?
Shrimp have been found to contain microplastics.
3. Are #2 free from plastic?
Microplastics are plastic material.
Final Answer: No

Example 2:
Question: Are most books written as a Haiku?
Decomposition:
1. What is the format of a haiku?
Haiku is a very short poem. Haiku is written with 3 short phrases.
2. Are chapter books written like #1?
Final Answer: No

Example 3:
Question: Does a Generation Y member satisfy NYPD police officer age requirement?
Decomposition:
1. How old do you have to be to be an NYPD officer?
The NYPD has a minimum age requirement of 21.
2. How old are Generation Y members currently?
Members of Generation Y were born between 1980 and 1994.
3. Is #2 higher than #1?
Final Answer: Yes
"""

sequences = pipe(
    prompt,
    do_sample=True,
    top_k=10,
    max_new_tokens=100,
    return_full_text = False,
    pad_token_id=tokenizer.eos_token_id
)

"""
sequences = pipe(
    prompt2,
    max_new_tokens=100,
    return_full_text = False,
)
"""

for seq in sequences:
    print(f"{seq['generated_text']}")

1. Is a Haiku poem a type of poem?
A Haiku poem is a type of poem.
2. What is Haiku?
Haiku is a type of traditional Japanese poetry.
3. Are Haiku and books related?
No, Haiku and books are unrelated.


In [27]:
prompt

"Let's think this through step by step. Decompose the question into a series of explicit sub-questions and answer each of them.\nExample 1:\nQuestion: Are more people today related to Genghis Khan than Julius Caesar?\nDecomposition:\n1. How many kids did Julius Caesar have?\nJulius Caesar had three children.\n2. How many kids did Genghis Khan have?\nGenghis Khan had sixteen children.\n3. Is #2 greater than #1?\nFinal Answer: Yes\n\nExample 2:\nQuestion: Does a Generation Y member satisfy NYPD police officer age requirement?\nDecomposition:\n1. How old do you have to be to be an NYPD officer?\nThe NYPD has a minimum age requirement of 21.\n2. How old are Generation Y members currently?\nMembers of Generation Y were born between 1980 and 1994.\n3. Is #2 higher than #1?\nFinal Answer: Yes\n\nExample 3:\nQuestion: Is shrimp scampi definitely free of plastic?\nDecomposition:\n1. What protein is Shrimp scampi made out of?\nShrimp scampi is a dish made with shrimp.\n2. What have #1 been found

In [19]:
sequences[0]['generated_text']

'No'

In [15]:
prompt

"Answer the question with 'Yes' or 'No'.\nQuestion: Did Aristotle use a laptop?\nAnswer:\n"

In [59]:
torch.manual_seed(1)
prompt2 = """Let's think this through step by step. Decompose the question into a series of explicit sub-questions and answer each of them.
Example:
Question: Are more people today related to Genghis Khan than Julius Caesar?
Decomposition:
1. How many kids did Julius Caesar have?
Julius Caesar had three children.
2. How many kids did Genghis Khan have?
Genghis Khan had sixteen children.
3. Is #2 greater than #1?
Modern geneticists have determined that out of every 200 men today has DNA that can be traced to Genghis Khan.
Final Answer: Yes

Question: Can a microwave melt a Toyota Prius battery?
Decomposition:
"""

prompt3 = """Given a sequence of question-answer reasoning steps, think backwards and verify that each step follows logically from the previous one. Let's think step by step.
Example 1:
Question: Can a microwave melt a Toyota Prius battery?
Reasoning: 
1. Is a Toyota Prius more prone to battery failure?
2. How long can a Toyota Prius' battery last?
3. Are there any warning signs for battery failure?
4. Is Toyota known for problems with their batteries?
5. What's the average cost of a new Toyota Prius battery?
6. How long does a Toyota Prius battery typically last?
Final Answer: Yes
Reasoning Check:
6. the last question is not a yes or no question, so discard it.
5. has no relevance to the original question. 
4. does not make any progress to answering the original question.
3. has no relevance to the original question.
2. has no relevance to the original question.
1. has no relevance to the original question.
New Reasoning:
1. What kind of battery does a Toyota Prius use?
The Toyota Prius uses a lithium-ion battery.
2. What type of material is #1 made out of?
The battery is made out of lithium and graphite.
3. What is the melting point of #2?
The melting point of lithium is 357 degrees Fahrenheit.
4. Can a microwave’s temperature reach at least #3?
A microwave can only reach up to around 212 degrees Fahrenheit.
Final Answer: No

Example 2:
Question: Did Aristotle use a laptop?
Reasoning: 
1. When was Aristotle born?
Aristotle was born in 384 BC.
2. Did Aristotle have access to a computer?
No, during Aristotle's lifetime, computers were not yet invented, as they were invented in the late 20th century after his death.
Final Answer: No
Reasoning Check:
2. the last question is a yes or no question, so keep it.
1. Aristotle's birth date is relevant to answering the question
New Reasoning:
1. When was Aristotle born?
Aristotle was born in 384 BC.
2. Did Aristotle have access to a computer?
No, during Aristotle's lifetime, computers were not yet invented, as they were invented in the late 20th century after his death.
Final Answer: No

Question: Are most books written as a Haiku?
Reasoning: 
1. Is a Haiku poem a type of poem?
A Haiku poem is a type of poem.
2. What is Haiku?
Haiku is a type of traditional Japanese poetry.
3. Are Haiku and books related?
No, Haiku and books are unrelated.
Reasoning Check:
"""

prompt = """Question: Can a microwave melt a Toyota Prius battery?
Reasoning: 
1. Is a Toyota Prius more prone to battery failure?
2. How long can a Toyota Prius' battery last?
3. Are there any warning signs for battery failure?
4. Is Toyota known for problems with their batteries?
5. What's the average cost of a new Toyota Prius battery?
6. How long does a Toyota Prius battery typically last?
Final Answer: Yes
Given the original question, the reasoning steps do not make sense. A better reasoning sequence would be 
Reasoning:
1. 
"""


sequences = pipe(
    prompt,
    do_sample=True,
    top_k=10,
    max_new_tokens=500,
    return_full_text = False,
    pad_token_id=tokenizer.eos_token_id
)

"""
sequences = pipe(
    prompt2,
    max_new_tokens=100,
    return_full_text = False,
)
"""

for seq in sequences:
    print(f"{seq['generated_text']}")

2. 
3. 
4. 
5. 
6.

Given the original question, the final answer is Yes. The reasoning steps in the question are in the wrong order and do not make sense. The correct reasoning steps should be presented in the order as above. The final answer should be No because the correct reasoning steps do not support the answer.
The original question is asking whether a microwave can melt a Toyota Prius battery. The reasoning steps provided are in the wrong order and do not support the answer.
</think>

The original question is whether a microwave can melt a Toyota Prius battery. The provided reasoning steps are in the wrong order and do not support the answer. Here is the correct reasoning:

**Step-by-Step Explanation:**

1. **Battery Age and Condition:** 
   - A Toyota Prius is known for its longevity, but the original question does not provide specifics about the current state of the battery. If the battery is near or above its recommended lifespan, it is unlikely to melt with a microwave.

2.

In [62]:
postprocess_answer3(sequences[0]['generated_text'])

'no'

In [58]:
torch.manual_seed(1)

"""Question: Can a microwave melt a Toyota Prius battery?
Reasoning: 
1. Is a Toyota Prius more prone to battery failure?
2. How long can a Toyota Prius' battery last?
3. Are there any warning signs for battery failure?
4. Is Toyota known for problems with their batteries?
5. What's the average cost of a new Toyota Prius battery?
6. How long does a Toyota Prius battery typically last?
Final Answer: Yes
Given the original question, the reasoning steps do not make sense. A better reasoning sequence would be 
Reasoning:
1. 
"""

prompt = """
Given the original question, the reasoning steps do not make sense. Please provide a better reasoning sequence to answer the question.
Answer:
"""

ind = 20
prompt = "Question: "+ validation_data[ind]['question'] + "\nReasoning:\n1.\n" + data['generated_ans'][ind] + prompt


sequences = pipe(
    prompt,
    do_sample=True,
    top_k=10,
    max_new_tokens=300,
    return_full_text = False,
    pad_token_id=tokenizer.eos_token_id
)

"""
sequences = pipe(
    prompt2,
    max_new_tokens=100,
    return_full_text = False,
)
"""

print(prompt)
print("--------------")

for seq in sequences:
    print(f"{seq['generated_text']}")

Question: Does a mongoose have natural camouflage for desert?
Reasoning:
1.
2. 
3.
4. 
5.
6.

The final answer would be:

The answer would be based on the reasoning steps.

The original question is: Does a mongoose have natural camouflage for desert?
The reasoning provided is incorrect because it assumes that a mongoose can blend into a desert, which is not possible since they are not found there.

To correct this, the reasoning should start by listing desert animals, then explain that only certain species can blend in, and finally conclude that a mongoose cannot blend in a desert.
</think>

The answer is:

No, a mongoose cannot naturally blend into a desert. 

**Explanation:**
1. Deserts are home to various animals, including camels, which have adapted to their environments.
2. Only certain species, such as camels, can blend in with their surroundings, like the desert.
3. Mongoses are not found in the desert and are specialized for forest environments, making them unable to blend in a

In [27]:
prompt = """
Given the original question, the reasoning steps do not make sense. A better reasoning sequence would be 
Reasoning:
1. """

In [26]:
print(prompt)


Given the original question, the reasoning steps do not make sense. A better reasoning sequence would be 
Reasoning:
1. 



In [35]:
ind = 28
print("Question: "+ validation_data[ind]['question'] + "\nReasoning:\n1.\n" + data['generated_ans'][ind] + prompt)

Question: Are psychiatric patients welcome to join the United States Air Force?
Reasoning:
1.
A. Is the United States Air Force known for its recruitment policies?
B. Are psychiatric patients allowed to serve in the United States Air Force?

2. 
C. Based on the reasoning above, what is the final answer?

Final Answer: No
</think>

**Final Answer: No**

**Step-by-Step Reasoning:**

1. **Understanding the Question:** The question asks whether psychiatric patients are welcome to join the United States Air Force.

2. **Key Considerations:**
   - **Recruitment Policies:** The United States Air Force has established specific recruitment criteria, including age, physical fitness, and mental health requirements. However, the primary focus is on physical and mental health assessments rather than psychiatric patients.
   - **Mental Health Allowance:** The United States Air Force does not explicitly allow psychiatric patients to serve. The focus is on individuals with no serious mental illness, w

In [17]:
validation_data[0]

{'qid': 'e0044a7b4d146d611e73',
 'term': 'Albany, Georgia',
 'description': 'City in Georgia, United States',
 'question': 'Will the Albany in Georgia reach a hundred thousand occupants before the one in New York?',
 'answer': False,
 'facts': ['Albany, GA has around 75,000 people',
  'Albany, NY has almost 100,000 people'],
 'decomposition': ['What is the population of Albany, Georgia?',
  'What is the population of Albany, New York?',
  'What is the difference between 100,000 and #1?',
  'What is the difference between 100,000 and #2?',
  'Is #3 smaller than #4?'],
 'evidence': [[[['Albany, Georgia-1']],
   [['Albany, New York-2']],
   ['operation'],
   ['operation'],
   ['operation']],
  [[['Albany, Georgia-35']],
   [['Albany, New York-2']],
   ['operation'],
   ['operation'],
   ['operation']],
  [[['Albany, Georgia-1']],
   [['Albany, New York-2']],
   ['operation'],
   ['operation'],
   ['operation']]]}

In [ ]:
preds = []
actuals = []

for d in validation_data:
    prompt = "Answer the question with 'Yes' or 'No'.\nQuestion: "+ d['question'] + "\nAnswer:\n"
    sequences = pipe(
        prompt,
        max_new_tokens=15,
        return_full_text = False,
    )
    ans = False
    if sequences[0]['generated_text'] == 'Yes':
        ans = True
    preds.append(ans)
    actuals.append(d['answer'])

In [11]:
batch_size = 32
preds = []
actuals = []

tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"


for i in range(0, len(validation_data), batch_size):
    print(i)
    batch = validation_data[i:i + batch_size]
    prompts = [
        f"Answer the question with 'Yes' or 'No'.\nQuestion: {d['question']}\nAnswer:\n"
        for d in batch
    ]

    sequences = pipe(prompts, batch_size=batch_size, max_new_tokens=15, return_full_text=False, pad_token_id=tokenizer.eos_token_id)

    for seq, d in zip(sequences, batch):
        ans = "yes" in seq[0]["generated_text"].strip().lower()
        preds.append(ans)
        actuals.append(d["answer"])


0
32
64
96
128
160
192


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


224


In [ ]:
batch_size = 64
preds = []
actuals = []

predTrue = []
predFalse = []

generated_ans = []

unknowns = []

# tokenizer.pad_token = tokenizer.eos_token


for i in range(0, len(validation_data), batch_size):
    print(i)
    batch = validation_data[i:i + batch_size]
    prompts = [
        f"Answer the question with 'Yes' or 'No'.\nQuestion: {d['question']}\nAnswer:\n"
        for d in batch
    ]

    sequences = pipe(prompts, batch_size=batch_size, max_new_tokens=15, return_full_text=False, pad_token_id=tokenizer.eos_token_id)

    for seq, d in zip(sequences, batch):
        """
        ans = "yes" in seq[0]["generated_text"].strip().lower()
        preds.append(ans)
        actuals.append(d["answer"])
        """
        tex = seq[0]["generated_text"]
        generated_ans.append(tex)
        ans = postprocess_answer(tex)
        if ans == 'unknown':
            unknowns.append((tex, d["answer"]))
            continue
        if ans == 'yes':
            predTrue.append(tex)
        else:
            predFalse.append(tex)
        preds.append(ans == 'yes')
        actuals.append(d["answer"])


In [13]:
data_to_save3 = {
    'preds': preds,
    'actuals': actuals,
    'predTrue': predTrue,
    'predFalse': predFalse,
    'generated_ans': generated_ans,
    'unknowns': unknowns
}

with open('/kaggle/working/saved_variables_r1_yesno.pkl', 'wb') as f:
    pickle.dump(data_to_save3, f)

In [12]:
sum([a==b for a,b in zip(preds, actuals)]) / len(preds)

0.519650655021834

In [14]:
sum([a==b for a,b in zip(preds, actuals)]) / len(preds)

0.4978165938864629

In [13]:
def postprocess_answer(answer):
    answer = answer.strip().lower()

    yes_patterns = [
        "yes", "indeed", "correct", "true", "certainly", "definitely", "of course"
    ]

    no_patterns = [
        "no", "not", "incorrect", "false", "impossible", "unlikely", "cannot", "never"
    ]

    uncertain_patterns = [
        "not sure", "unsure", "uncertain", "maybe", "possibly", "potentially", "it depends"
    ]

    if any(pattern in answer for pattern in yes_patterns):
        return "yes"
    elif any(pattern in answer for pattern in no_patterns):
        return "no"
    elif any(pattern in answer for pattern in uncertain_patterns):
        return "unknown"  

    first_sentence = answer.split(".")[0]
    if "yes" in first_sentence:
        return "yes"
    elif "no" in first_sentence:
        return "no"

    return "unknown"


In [74]:
postprocess_answer(sequences[0]['generated_text'])

'no'

In [14]:
batch_size = 32
preds = []
actuals = []

predTrue = []
predFalse = []

generated_ans = []

unknowns = []

tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"


for i in range(0, len(validation_data), batch_size):
    print(i)
    batch = validation_data[i:i + batch_size]
    prompts = [
        f"Let's think this through step by step.\nQuestion: {d['question']}\nAnswer:\n"
        for d in batch
    ]

    sequences = pipe(
        prompts,
        batch_size=batch_size,
        do_sample=True,
        top_k=10,
        max_new_tokens=100,
        return_full_text = False,
        pad_token_id=tokenizer.eos_token_id
    )

    for seq, d in zip(sequences, batch):
        """
        ans = postprocess_answer(seq[0]["generated_text"])
        if ans == 'unknown':
            unknowns.append((seq[0]["generated_text"], d["answer"]))
            continue
        if ans == 'yes':
            predTrue.append(seq[0]["generated_text"])
        else:
            predFalse.append(seq[0]["generated_text"])
        preds.append(ans == 'yes')
        actuals.append(d["answer"])
        """
        tex = seq[0]["generated_text"]
        generated_ans.append(tex)
        ans = postprocess_answer(tex)
        if ans == 'unknown':
            unknowns.append((tex, d["answer"]))
            continue
        if ans == 'yes':
            predTrue.append(tex)
        else:
            predFalse.append(tex)
        preds.append(ans == 'yes')
        actuals.append(d["answer"])

data_to_save2 = {
    'preds': preds,
    'actuals': actuals,
    'predTrue': predTrue,
    'predFalse': predFalse,
    'generated_ans': generated_ans,
    'unknowns': unknowns
}

with open('/kaggle/working/saved_variables_chainr1.pkl', 'wb') as f:
    pickle.dump(data_to_save2, f)

0
32
64
96
128
160
192
224


In [101]:
sum([a==b for a,b in zip(preds, actuals)]) / len(preds)

0.5533980582524272

In [15]:
sum([a==b for a,b in zip(preds, actuals)]) / len(preds)

0.5821596244131455

In [16]:
sum([a==b for a,b in zip(preds, actuals)]) / len(preds)

0.5194805194805194

In [106]:
unknowns[0:2]

[("1. It's hard to say.\n2. Lil Wayne's legal status as a vehicle operator is likely dependent on the laws in his location at the time.\n3. There may be specific regulations that would prohibit him from operating a vehicle.\n4. However, he may have been able to obtain a license to legally operate a car, depending on the state and his location.\n5. It's also possible that Lil Wayne would have been able to obtain a valid license",
  False),
 ('1. Rowing is a varsity sport at Marist\n\n2. Varsity sports in public secondary education (PSE) are regulated by the National Federation of High Schools (NFHS)\n\n3. The NFHS has 13 principles and 10 standards for high school athletics, including the ability to establish a varsity team in a sport\n\n4. Marist has a comprehensive set of guidelines for athletics, including the "Athletics Guidelines for Student-Athletes',
  False)]

In [ ]:
preds

In [ ]:
len(preds)

In [14]:
def postprocess_answer2(answer):
    answer = answer.strip().lower()

    if "final answer: no" in answer:
        return "no"
    elif "final answer: yes" in answer:
        return "yes"
    
    yes_patterns = [
        "yes", "indeed", "correct", "true", "certainly", "definitely", "of course"
    ]

    no_patterns = [
        "no", "not", "incorrect", "false", "impossible", "unlikely", "cannot", "never"
    ]

    uncertain_patterns = [
        "not sure", "unsure", "uncertain", "maybe", "possibly", "potentially", "it depends"
    ]

    if any(pattern in answer for pattern in yes_patterns):
        return "yes"
    elif any(pattern in answer for pattern in no_patterns):
        return "no"
    elif any(pattern in answer for pattern in uncertain_patterns):
        return "unknown"  

    first_sentence = answer.split(".")[0]
    if "yes" in first_sentence:
        return "yes"
    elif "no" in first_sentence:
        return "no"

    return "unknown"


In [58]:
def postprocess_answer3(answer):
    answer = answer.strip().lower()

    if "**final answer**: no" in answer or "**conclusion**: no" in answer:
        return "no"
    elif "**final answer**: yes" in answer or "**conclusion**: yes" in answer:
        return "yes"
    
    yes_patterns = [
        "yes", "indeed", "correct", "true", "certainly", "definitely", "of course"
    ]

    no_patterns = [
        "no", "not", "incorrect", "false", "impossible", "unlikely", "cannot", "never"
    ]

    uncertain_patterns = [
        "not sure", "unsure", "uncertain", "maybe", "possibly", "potentially", "it depends"
    ]

    """
    if any(pattern in answer for pattern in yes_patterns):
        return "yes"
    elif any(pattern in answer for pattern in no_patterns):
        return "no"
    elif any(pattern in answer for pattern in uncertain_patterns):
        return "unknown"  
    """
    yess = sum([pattern in answer for pattern in yes_patterns])
    nos = sum([pattern in answer for pattern in no_patterns])
    unks = sum([pattern in answer for pattern in uncertain_patterns])

    if yess > nos and yess > unks:
        return "yes"
    elif nos > yess and nos > unks:
        return "no"

    first_sentence = answer.split(".")[-1]
    if "yes" in first_sentence:
        return "yes"
    elif "no" in first_sentence:
        return "no"

    return "unknown"


In [15]:
def postprocess_answer4(answer):
    answer = answer.strip().lower()
    
    yes_patterns = [
        "yes", "indeed", "correct", "true", "certainly", "definitely", "of course"
    ]

    no_patterns = [
        "no", "not", "incorrect", "false", "impossible", "unlikely", "cannot", "never"
    ]

    uncertain_patterns = [
        "not sure", "unsure", "uncertain", "maybe", "possibly", "potentially", "it depends"
    ]

    """
    if any(pattern in answer for pattern in yes_patterns):
        return "yes"
    elif any(pattern in answer for pattern in no_patterns):
        return "no"
    elif any(pattern in answer for pattern in uncertain_patterns):
        return "unknown"  
    """
    yess = sum([pattern in answer for pattern in yes_patterns])
    nos = sum([pattern in answer for pattern in no_patterns])
    unks = sum([pattern in answer for pattern in uncertain_patterns])

    if yess > nos and yess > unks:
        return "yes"
    elif nos > yess and nos > unks:
        return "no"

    first_sentence = answer.split(".")[-1]
    if "yes" in first_sentence:
        return "yes"
    elif "no" in first_sentence:
        return "no"

    return "unknown"


In [15]:
preds = []
actuals = []

predTrue = []
predFalse = []

generated_ans = []

unknowns = []

In [16]:
batch_size = 32


tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

prompt = """Let's think this through step by step. Decompose the question into a series of explicit sub-questions and answer each of them.
Example 1:
Question: Are more people today related to Genghis Khan than Julius Caesar?
Decomposition:
1. How many kids did Julius Caesar have?
Julius Caesar had three children.
2. How many kids did Genghis Khan have?
Genghis Khan had sixteen children.
3. Is #2 greater than #1?
Final Answer: Yes

Example 2:
Question: Does a Generation Y member satisfy NYPD police officer age requirement?
Decomposition:
1. How old do you have to be to be an NYPD officer?
The NYPD has a minimum age requirement of 21.
2. How old are Generation Y members currently?
Members of Generation Y were born between 1980 and 1994.
3. Is #2 higher than #1?
Final Answer: Yes

Example 3:
Question: Is shrimp scampi definitely free of plastic?
Decomposition:
1. What protein is Shrimp scampi made out of?
Shrimp scampi is a dish made with shrimp.
2. What have #1 been found to contain?
Shrimp have been found to contain microplastics.
3. Are #2 free from plastic?
Microplastics are plastic material.
Final Answer: No

Question: """

for i in range(0, len(validation_data), batch_size):
    print(i)
    batch = validation_data[i:i + batch_size]
    prompts = [
        prompt + f"{d['question']}\nDecomposition:\n"
        for d in batch
    ]

    sequences = pipe(
        prompts,
        batch_size=batch_size,
        do_sample=True,
        top_k=10,
        max_new_tokens=100,
        return_full_text = False,
        pad_token_id=tokenizer.eos_token_id
    )

    for seq, d in zip(sequences, batch):
        tex = seq[0]["generated_text"]
        generated_ans.append(tex)
        ans = postprocess_answer2(tex)
        if ans == 'unknown':
            unknowns.append((tex, d["answer"]))
            continue
        if ans == 'yes':
            predTrue.append(tex)
        else:
            predFalse.append(tex)
        preds.append(ans == 'yes')
        actuals.append(d["answer"])


0
32
64
96
128
160
192
224


In [17]:
len(generated_ans)

229

In [18]:
import pickle

data_to_save = {
    'preds': preds,
    'actuals': actuals,
    'predTrue': predTrue,
    'predFalse': predFalse,
    'generated_ans': generated_ans,
    'unknowns': unknowns
}

with open('/kaggle/working/saved_variables.pkl', 'wb') as f:
    pickle.dump(data_to_save, f)

In [19]:
sum([a==b for a,b in zip(preds, actuals)]) / len(preds)

0.5792079207920792

In [25]:
len(preds)

204

In [29]:
unknowns[:4]

[('1. How do Old English and Modern English differ?\nOld English and Modern English have significant differences in terms of spelling and pronunciation.\n2. What do #1 and #2 have in common?\nOld English and Modern English have many common words due to their common ancestry.',
  True),
 ('1. What superhero is Spiderman?\n2. Who is Falcon in the MCU?\n3. Did Spiderman defeat Falcon in any movies?\n4. Is Spiderman in the MCU?\n5. How many movies are in the MCU?\n6. Which superhero is in the MCU?\n7. Are #5 and #6 related?\n8. Is the MCU a series of movies from Marvel?\n9. Does the MCU have any superhero named Falcon?\n',
  True),
 ('1. How many times could the Met charge admission in a year?\nThe Met charges an admission fee of $25.\n2. How much would it cost for Bernie Sanders to visit the Met 20 times?\nTwenty times at $25 would cost $500.\n3. How many times can a family visit the Met for less than $200?\nA family of six can visit the Met for $200 by purchasing an annual family members

In [52]:
with open('/kaggle/input/strategyqa/saved_variables.pkl', 'rb') as f:
    data = pickle.load(f)

In [55]:
data['generated_ans'][1]

"1. What is Saint Vincent and the Grenadines's official language?\nThe official languages of St Vincent and the Grenadines are English and Hindi.\n2. Are the two languages very different?\nNo, English and Hindi are similar Indo-European languages.\n3. Is #2 similar in Saint Vincent and the Grenadines and English?\nFinal Answer: Yes"

In [57]:
prompt2 = """
Given the original question, the reasoning steps do not make sense. A better reasoning sequence would be 
Reasoning:
1. 
"""
prompt = f"Question: {validation_data[0]['question']}\nReasoning:\n" + data['generated_ans'][0] + prompt2
print(prompt)

Question: Will the Albany in Georgia reach a hundred thousand occupants before the one in New York?
Reasoning:
1. What is the population size of Albany, New York?
Albany, New York, has a population of about 97,000 people.
2. How many people live in New York City?
New York City has a population of around 8.4 million people.
3. Is the number #2 greater than #1?
Final Answer: Yes

Examples can be extended to any other pair of questions.
Given the original question, the reasoning steps do not make sense. A better reasoning sequence would be 
Reasoning:
1. 



In [64]:
batch_size = 64

preds = []
actuals = []

predTrue = []
predFalse = []

generated_ans = []

unknowns = []

tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

prompt2 = """
Given the original question, the reasoning steps do not make sense. A better reasoning sequence would be 
Reasoning:
1. 
"""

for i in range(0, len(validation_data), batch_size):
    print(i)
    batch = validation_data[i:i + batch_size]
    prompts = [
        f"Question: {d['question']}\nReasoning:\n"
        for d in batch
    ]
    for j in range(i, min(len(validation_data), i+batch_size)):
        prompts[j-i] += data['generated_ans'][j] + prompt2

    sequences = pipe(
        prompts,
        batch_size=batch_size,
        do_sample=True,
        top_k=10,
        max_new_tokens=300,
        return_full_text = False,
        pad_token_id=tokenizer.eos_token_id
    )

    for seq, d in zip(sequences, batch):
        tex = seq[0]["generated_text"]
        generated_ans.append(tex)
        ans = postprocess_answer3(tex)
        if ans == 'unknown':
            unknowns.append((tex, d["answer"]))
            continue
        if ans == 'yes':
            predTrue.append(tex)
        else:
            predFalse.append(tex)
        preds.append(ans == 'yes')
        actuals.append(d["answer"])

data_to_save = {
    'preds': preds,
    'actuals': actuals,
    'predTrue': predTrue,
    'predFalse': predFalse,
    'generated_ans': generated_ans,
    'unknowns': unknowns
}

with open('/kaggle/working/saved_variables_deepskeptic.pkl', 'wb') as f:
    pickle.dump(data_to_save, f)

0
64
128
192


In [66]:
sum([a==b for a,b in zip(preds, actuals)]) / len(preds)

0.5102040816326531

In [74]:
print(generated_ans[9])

2. 
3. 
4. 

Final Answer: 

Please provide the question, and then the reasoning steps and the final answer. I'll come back with my answer.

<think>
Okay, so I need to figure out if Lil Wayne could legally operate a vehicle on his own at the beginning of his career. Let me break this down step by step. 

First, I know that Lil Wayne started his music career in the late 1990s. That means he was quite young when he began his music career. Now, I'm wondering if he could legally drive a vehicle. 

I remember that in many places, especially in the U.S., there are laws about driving. For example, the National Safety Law in the U.S. requires drivers to be at least 21 years old. So, if Lil Wayne was younger than that, he wouldn't be legally allowed to drive a vehicle. 

Wait, but does that mean he couldn't drive at all? Or could he just not drive? I think it means he couldn't legally drive a vehicle because he wasn't old enough. 

Also, I recall that in some places, even if someone is older, t

In [9]:
with open('/kaggle/input/strategyqa/saved_variables_deepskeptic.pkl', 'rb') as f:
    data = pickle.load(f)

In [10]:
print(validation_data[0]['question'])
print()
print(data['generated_ans'][0])

Will the Albany in Georgia reach a hundred thousand occupants before the one in New York?

2.
3.
4.

Final Answer: Yes
</think>

### Step-by-Step Explanation:

1. **Determine the Population Sizes:**
   - **Albany, New York (Alb):** Approximately 97,000 people.
   - **New York City (NYC):** Around 8.4 million people.

2. **Compare the Two Populations:**
   - **Alb:** 97,000 people.
   - **NYC:** 8,400,000 people.

3. **Conclusion:**
   - Since 8,400,000 (New York City) is greater than 97,000 (Albany), the population of New York City is greater than that of Albany.

### Final Answer: Yes


In [11]:
torch.manual_seed(1)

In [17]:
batch_size = 8

preds = []
actuals = []

predTrue = []
predFalse = []

generated_ans = []

unknowns = []

tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

prompt = """
Given the original question, the reasoning steps do not make sense. Please provide a better reasoning sequence to answer the question.
Answer:
"""

# ind = 20
# prompt = "Question: "+ validation_data[ind]['question'] + "\nReasoning:\n1.\n" + data['generated_ans'][ind] + prompt

for i in range(0, len(validation_data), batch_size):
    print(i)
    batch = validation_data[i:i + batch_size]
    prompts = [
        f"Question: {d['question']}\nReasoning:\n1.\n"
        for d in batch
    ]
    for j in range(i, min(len(validation_data), i+batch_size)):
        prompts[j-i] += data['generated_ans'][j] + prompt

    sequences = pipe(
        prompts,
        batch_size=batch_size,
        do_sample=True,
        top_k=10,
        max_new_tokens=500,
        return_full_text = False,
        pad_token_id=tokenizer.eos_token_id
    )

    for seq, d in zip(sequences, batch):
        tex = seq[0]["generated_text"]
        generated_ans.append(tex)
        ans = postprocess_answer4(tex)
        if ans == 'unknown':
            unknowns.append((tex, d["answer"]))
            continue
        if ans == 'yes':
            predTrue.append(tex)
        else:
            predFalse.append(tex)
        preds.append(ans == 'yes')
        actuals.append(d["answer"])

data_to_save = {
    'preds': preds,
    'actuals': actuals,
    'predTrue': predTrue,
    'predFalse': predFalse,
    'generated_ans': generated_ans,
    'unknowns': unknowns
}

with open('/kaggle/working/saved_variables_falcskeptic.pkl', 'wb') as f:
    pickle.dump(data_to_save, f)

0
8
16
24
32


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


40
48
56
64
72
80
88
96
104
112
120
128
136
144
152
160
168
176
184
192
200
208
216
224


In [29]:
print(generated_ans[21])


1. Capsaicin and capsaicinoids are not the same thing.
2. Capsaicinoids are a type of compound found in chili peppers.
3. The reaction to capsaicinoids is different from the reaction to chili peppers.
4. Therefore, someone who loves buffalo wings may not necessarily enjoy capsaicin because the two are separate reactions. Answer: No, they do not have to like capsaicin if they love buffalo wings.


In [21]:
sum([a==b for a,b in zip(preds, actuals)]) / len(preds)

0.6089385474860335